In [33]:
# Importing necessary libraries

# Nearest neighbors will be the v1 model for sound drip
# will give logistic regressiona nd 
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from joblib import dump
from joblib import load
import pickle
import numpy as np

import pandas as pd
from pandas.io.json import json_normalize

# textblob will be used to perform sentiment analysis down the road
# from textblob import TextBlob

# # Data is large and will need to be unzipped
# import zipfile

# # unzipping file
# with zipfile.ZipFile("./data_collection/final_data/DF_v1.pkl.zip","r") as zip_ref:
#     zip_ref.extractall(".")

# # reading in pickled data
# song_list = pd.read_pickle("./DF_v1.pkl")

In [ ]:
# ALERT - THIS COULD POTENTIALLY BE USED TO PERFORM SENTIMENT ANALYSIS ON LYRICS DATA IN v2. FOR v1, DISREGARD

# # creating polarity and subjectivity lists to be populated

# lyrics_polarity = []
# lyrics_subjectivity = []

# # using a for loop to populate polarity and subjectivity lists
# for title in song_list_df["lyrics"]:
#     try:
#         blob_test = TextBlob = TextBlob(title)
#         blob_sentiment = blob_test.sentiment
#         # appending results to lists above
#         lyrics_polarity.append(blob_sentiment[0])
#         lyrics_subjectivity.append(blob_sentiment[1])
#     except:
#         lyrics_polarity.append(0)
#         lyrics_subjectivity.append(0)

# # adding polarity and subjectivity to df
# song_list['lyrics_polarity'] = lyrics_polarity
# song_list['lyrics_subjectivity'] = lyrics_subjectivity

# Cleaning/Scaling Data

In [11]:
# song_list = pickle.load("./DF_genres_hashed.pkl","rb")

song_list = pickle.load( open( "DF_genres_hashed.pkl", "rb"))

In [12]:
song_list.shape, song_list.columns

((574018, 68),
 Index([    'acousticness',           'artist',     'danceability',
             'duration_ms',           'energy', 'instrumentalness',
                     'key',         'liveness',         'loudness',
                    'mode',           'songid',      'speechiness',
                   'tempo',   'time_signature',            'track',
                 'valence',           'genres',  'genres_stripped',
                         0,                  1,                  2,
                         3,                  4,                  5,
                         6,                  7,                  8,
                         9,                 10,                 11,
                        12,                 13,                 14,
                        15,                 16,                 17,
                        18,                 19,                 20,
                        21,                 22,                 23,
                        24,      

In [13]:
# dropping unnecessary columns
X = song_list.drop(["songid","artist","track", "duration_ms","genres","genres_stripped"], axis=1)

# checking that they're dropped
X.shape

(574018, 62)

In [20]:
columns_for_scaling = X.columns[0:12]

In [25]:
columns_genre_hashed = X.columns[13:]

In [21]:
X[columns_for_scaling]

,acousticness,danceability,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence
0,0.001300,0.487,0.678,0.055100,9.0,0.0846,-7.780,1.0,0.0495,149.940,4.0,0.8700
1,0.000045,0.662,0.823,0.952000,4.0,0.3430,-1.711,0.0,0.0662,177.745,4.0,0.6210
2,0.002760,0.859,0.741,0.000000,11.0,0.3250,-12.329,0.0,0.2710,98.082,4.0,0.5290
3,0.348000,0.361,0.483,0.000003,7.0,0.1770,-6.875,1.0,0.0287,94.538,4.0,0.6820
4,0.340000,0.533,0.302,0.179000,10.0,0.1110,-10.308,1.0,0.0307,134.959,3.0,0.2940
...,...,...,...,...,...,...,...,...,...,...,...,...
574013,0.798000,0.320,0.104,0.000750,0.0,0.1060,-13.162,1.0,0.0342,124.677,3.0,0.0566
574014,0.820000,0.418,0.204,0.936000,8.0,0.0866,-13.522,1.0,0.0370,110.022,4.0,0.0375
574015,0.070400,0.614,0.952,0.000000,6.0,0.3620,-2.782,1.0,0.0718,148.079,4.0,0.8200
574016,0.064000,0.343,0.487,0.000000,2.0,0.1070,-9.462,1.0,0.0473,171.217,4.0,0.3650


In [26]:
X[columns_genre_hashed]

,1,2,3,4,5,6,7,8,9,10,...,40,41,42,43,44,45,46,47,48,49
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,...,0.0,-1.0,1.0,2.0,-3.0,0.0,0.0,0.0,0.0,-2.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,0.0,...,0.0,-4.0,5.0,7.0,-4.0,-1.0,0.0,0.0,0.0,-7.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,-3.0,0.0,1.0,-2.0,-1.0,0.0,0.0,0.0,-1.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,...,0.0,-2.0,1.0,2.0,-1.0,2.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,...,0.0,0.0,0.0,1.0,-1.0,0.0,0.0,0.0,0.0,-2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
574013,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,11.0,0.0,...,0.0,-6.0,1.0,9.0,-5.0,4.0,0.0,0.0,0.0,-1.0
574014,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,-2.0
574015,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,...,0.0,-2.0,3.0,6.0,-4.0,5.0,0.0,0.0,0.0,0.0
574016,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.0,0.0,...,0.0,-6.0,2.0,10.0,-5.0,7.0,0.0,0.0,0.0,-1.0


In [22]:
# instantiating scaler
scaler = MinMaxScaler()

# scaling data
data_scaled = scaler.fit_transform(X[columns_for_scaling])

# confirming scaling took place
data_scaled[0]

array([0.00130522, 0.4884654 , 0.678     , 0.0551    , 0.81818182,
       0.0846    , 0.78720453, 1.        , 0.05118925, 0.60009846,
       0.8       , 0.87      ])

In [30]:
# convert hashed genre columns to np.array

genres_hashed_array = pd.DataFrame.to_numpy(X[columns_genre_hashed])

In [31]:
len(genres_hashed_array)

574018

In [38]:
data_for_model = np.concatenate((data_scaled,genres_hashed_array),axis=1)

Data is scaled and ready to be fed to the model

# K-Nearest Neighbors Classifier w/ Hashed Genres

In [97]:
# instantiating model class
knn1 = NearestNeighbors(n_neighbors=100,algorithm='kd_tree')

# fitting model
model_1 = knn1.fit(data_for_model)

In [98]:
# running model to test output
knn_results1 = knn1.kneighbors([data_for_model[1005]])[1]

In [99]:
knn_results1[0]

array([  1005,  34253,  81221, 377832, 427172, 378705,  32006,  63622,
        34006,  74816, 244696, 396964, 371537, 137307,  60516, 103065,
       419260, 296547, 397450, 135433, 469512,  71563, 292610, 408450,
       470525, 472914, 281596, 412798, 299438, 233269, 355640, 397767,
        13176, 258222,  76127, 408068, 316140, 405485,    392, 365258,
       426022, 282527,  55323, 275270, 221856,  72070, 336924, 375194,
       153085, 409416, 441157, 324990, 134669, 313840,  96060, 216509,
       169409, 388634,  30562, 206745, 140862, 430124, 433466, 306928,
       390915, 265989,  29267, 430103, 114602, 276809, 161032, 324417,
       261442,  49037, 253997, 400818, 372333,  74893, 317634,  46054,
       172398, 354746, 437981,  47011, 323475, 404726,  59743, 441744,
       210913, 221382, 421387, 451475,  80889, 529929, 529930,   6808,
        81120, 302444,  33721,  89042])

In [100]:
pd.DataFrame(song_list.loc[1005][1:11])

,1005
artist,Jean-michel Rotin
danceability,0.64
duration_ms,256133
energy,0.784
instrumentalness,0.0163
key,1
liveness,0.0355
loudness,-9.452
mode,1
songid,1uR9ohf3yyH1nygTAqqksv


In [101]:
for result in knn_results1[0]:
    display(pd.DataFrame(song_list.loc[result][10:11]),pd.DataFrame(song_list.loc[result][17:18]))

,1005
songid,1uR9ohf3yyH1nygTAqqksv


,1005
genres_stripped,"'musique guadeloupe', 'zouk', 'zouk riddim'"


,34253
songid,2pgSJvvbp1vEb8C2phFGTU


,34253
genres_stripped,"'musique guadeloupe', 'zouk', 'zouk riddim'"


,81221
songid,2ZR44JNsEosu9IrG5yHzjx


...

,302444
genres_stripped,"'musique guadeloupe', 'rap kreyol', 'zouk', 'z..."


,33721
songid,0RWAPEgHVA6VW3TLOSlT5U


,33721
genres_stripped,"'musique guadeloupe', 'rap kreyol', 'zouk', 'z..."


,89042
songid,53A7wuZAIkESJEOWhKipBy


,89042
genres_stripped,"'musique guadeloupe', 'rap kreyol', 'zouk', 'z..."


# Model w/out Hashed Genres

In [92]:
# instantiating model class
knn2 = NearestNeighbors(n_neighbors=21,algorithm='kd_tree')

# fitting model
model2 = knn2.fit(data_scaled)

In [93]:
# running model to test output
knn_results2 = knn2.kneighbors([data_scaled[1005]])[1]

In [94]:
knn_results2[0]

array([  1005,  76243, 174154, 194031, 294534, 295656, 379547, 246778,
       147142, 470543, 282037, 208254, 472882, 252426, 397547,  51407,
       336683,  94372,  58406, 143793,  93618])

In [95]:
pd.DataFrame(song_list.loc[1005][1:11])

,1005
artist,Jean-michel Rotin
danceability,0.64
duration_ms,256133
energy,0.784
instrumentalness,0.0163
key,1
liveness,0.0355
loudness,-9.452
mode,1
songid,1uR9ohf3yyH1nygTAqqksv


In [96]:
for result in knn_results2[0]:
    display(pd.DataFrame(song_list.loc[result][10:11]),pd.DataFrame(song_list.loc[result][17:18]))

,1005
songid,1uR9ohf3yyH1nygTAqqksv


,1005
genres_stripped,"'musique guadeloupe', 'zouk', 'zouk riddim'"


,76243
songid,5GrMVumWvG1jgo44MY8Vei


,76243
genres_stripped,"'canadian indie', 'post-disco'"


,174154
songid,4yIAv4kJd7RI7CZTTSEJmy


...

,58406
genres_stripped,"'adult standards', 'brill building pop', 'brit..."


,143793
songid,1v0A4SYQEgYMiNGG2GxuKj


,143793
genres_stripped,'hard glam'


,93618
songid,0vsRt9W2VisbZW6mqqjbp8


,93618
genres_stripped,"'dance rock', 'mellow gold', 'new romantic', '..."


# Pickling model

In [8]:
import pickle

filename = './models/knn_model_v1.pkl'
pickle.dump(model, open(filename, 'wb'))

In [9]:
# pulling in the (now) pickled model
model_pickle = pickle.load(open(filename, 'rb'))

In [10]:
# Loading model to test on scaled data (not pickled data)
pickle_results = model_pickle.kneighbors([data_scaled[1000]])[1]

# displaying output
pickle_results

array([[  1000, 647630, 304823, 412399,  48174, 411195, 240739, 537833,
         50765, 450355, 151925, 707048, 557285, 508280, 175165, 510531,
        652499, 267268, 161932, 587960,  79755]])

# Pickling Scaled Data

In [11]:
filename2 = './data_scaled.pkl'
pickle.dump(data_scaled, open(filename2, 'wb'))

# (OLD) Testing both pickles for dataset, model

In [12]:
# importing libraries (even though they are visible above)
# to illustrate which packages are needed to create the function below

import pandas as pd 
from sklearn.neighbors import NearestNeighbors
import pickle


# this function predicts on a song id

def predict(id):
    #loads in pickled data and model
    X = pickle.load(open('./data_scaled.pkl', 'rb'))
    loaded_pickle = pickle.load(open('./models/knn_model_v1.pkl', 'rb'))
    #calculates results
    results = loaded_pickle.kneighbors([X[id]])[1]
    return results

In [13]:
predict(0)

array([[     0, 341644, 462704, 499704, 105639, 347586,  27005, 553442,
        594846, 273222, 636137, 218622, 375716,  40341, 510427, 518171,
        209537, 168110, 453579, 456931,  89949]])

In [14]:
content = {
    "audio_features": {
        "acousticness": 0.934,
        "danceability": 0.186,
        "energy": 0.107,
        "instrumentalness": 0,
        "key": 5,
        "liveness": 0.297,
        "loudness": -14.802,
        "mode": 1,
        "speechiness": 0.0347,
        "tempo": 107.095,
        "time_signature": 4,
        "valence": 0.149
    }
}

In [15]:
dataframe = pd.DataFrame.from_dict(
        json_normalize(content['audio_features']),
                                orient='columns')

In [16]:
dataframe

,acousticness,danceability,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence
0,0.934,0.186,0.107,0,5,0.297,-14.802,1,0.0347,107.095,4,0.149


In [17]:
dataframe_scaled = scaler.transform(dataframe)

dataframe_scaled

array([[0.937751  , 0.18655968, 0.107     , 0.        , 0.45454545,
        0.297     , 0.68134949, 1.        , 0.03584711, 0.42840913,
        0.8       , 0.149     ]])

In [22]:
import pickle

filename = './models/scaler.pkl'
pickle.dump(scaler, open(filename, 'wb'))

In [14]:
target = 'songid'

song_id_array = song_list[target]

In [15]:
song_id_array

0         5PS5dpaLogPzYU9hWiWyZb
1         41RpZW2lxAdnqDd2nMBzLQ
2         2poHURuOfVNbzZdivAwtOH
3         1jg9hZnReygpBvV2axGuPy
4         3GsS8jzoixpCnp4jDWCEvb
                   ...          
116091    1xm8J6EFMA6N8JDqH8vzuz
116238    4NwmHBjPb9i9N3naLCMVCG
116261    5Xo8AsEz0gpW6Rpo2jXvBN
116308    4Fnz6vDqufd3ens0Gf9LC5
116324    2tolmRzbUfgL5KRplIqHlu
Name: songid, Length: 728156, dtype: object

In [16]:
sys.getsizeof(song_id_array)

NameError: name 'sys' is not defined

In [17]:
song_id_list = song_id_array.tolist()

In [37]:
sys.getsizeof(song_id_list)

5825312

In [39]:
import pickle

filename = 'Flask_API/SOUNDDRIP/data/song_id_array.pkl'
pickle.dump(song_id_array, open(filename, 'wb'))

In [30]:
import pickle

filename = 'song_id_list2.pkl'
pickle.dump(song_id_list, open(filename, 'wb'))

In [3]:
content = {
    "audio_features": {
        "acousticness": 0.934,
        "danceability": 0.186,
        "energy": 0.107,
        "instrumentalness": 0,
        "key": 5,
        "liveness": 0.297,
        "loudness": -14.802,
        "mode": 1,
        "speechiness": 0.0347,
        "tempo": 107.095,
        "time_signature": 4,
        "valence": 0.149
    }
}

def predict(content):
    print('Loading dataframe...')
    dataframe = pd.DataFrame.from_dict(
        json_normalize(content['audio_features']),
                                orient='columns')
    print('Dataframe Object Created')
    print('Loading pickled scaler...')
    scaler = pickle.load(open('models/scaler.pkl', 'rb'))
    print('Pickled scaler loaded')
    print('Scaling dataframe object...')
    dataframe_scaled = scaler.transform(dataframe)
    print('Dataframe scaled')
    print('Loading pickled model...')
    model = pickle.load(open('./models/knn_model_v1.pkl', 'rb'))
    print('Model loaded')
    results = model.kneighbors([dataframe_scaled][0])[1]
    print('Prediction executed')

    return results


In [4]:
predict(content)

Loading dataframe...
Dataframe Object Created
Loading pickled scaler...


NameError: name 'pickle' is not defined

In [127]:
  # For-loop for returning 20 songs
def all_similarities(data_result):
    similar_songs = []
    print('song_id_list loading...')
    song_id_list = pickle.load(open('Flask_API/SOUNDDRIP/data/song_id_list.pkl', 'rb'))
    print('song_id_list loaded')
    print('beginning for loop...')
    
    for song_row in data_result[0][1:]:
        song_id = song_id_list[song_row]
        similar_songs.append({'similarity': [.99], 'values': song_id})
    json_dict = {"songs": similar_songs}
    return json_dict

In [128]:
all_similarities(predict(content))

Loading dataframe...
Dataframe Object Created
Loading pickled scaler...
Pickled scaler loaded
Scaling dataframe object...
Dataframe scaled
Loading pickled model...
Model loaded
Prediction executed
song_id_list loading...
song_id_list loaded
beginning for loop...


{'songs': [{'similarity': [0.99], 'values': '3JeSeOxS09vw7ZaQXyDtkn'},
  {'similarity': [0.99], 'values': '0lkEvsJLWu7HjurNUnMYqt'},
  {'similarity': [0.99], 'values': '7GrCeh9O4Salm3oEoiDWmf'},
  {'similarity': [0.99], 'values': '705JQ49YZOupAYQXrny5Vb'},
  {'similarity': [0.99], 'values': '7v6maEWDjrKHQMVFoL8Tq3'},
  {'similarity': [0.99], 'values': '6pJeuRa8dj6Hph2r9Unn5H'},
  {'similarity': [0.99], 'values': '1XsXX5Q8mCdlDjroYAUs8r'},
  {'similarity': [0.99], 'values': '6nl7z8eqCRRNSPJevUg5yJ'},
  {'similarity': [0.99], 'values': '4Swt07x4GyojZApyerQ8Ta'},
  {'similarity': [0.99], 'values': '68PGQTrsU7NTe6hfcFD3Fh'},
  {'similarity': [0.99], 'values': '5jTnsVnWuxdSQnRv124hwL'},
  {'similarity': [0.99], 'values': '5YsyWAtEDsStifCxF5d7wJ'},
  {'similarity': [0.99], 'values': '4RKMHiQ1sQCJVcPEbqcXR2'},
  {'similarity': [0.99], 'values': '7dXTqzYBW6LcuMvfmhKjeS'},
  {'similarity': [0.99], 'values': '1pQPfwF8yZejKeu2rZOIwI'},
  {'similarity': [0.99], 'values': '2ydHMFR78xJCTaUSgQsY1d'},

In [111]:
sorted_list = sorted(similarities, key=lambda i: i['similarity'], 
                                                         reverse=True)[:20]
      json_dict = {"songs": sorted_list}
      #data = json.dumps(json_dict)
      return jsonify(json_dict), print('yay')

'70jVRf1KHVa4eROjpdmaja'

# Get Spotify API Token / Output Audio Features

In [1]:
import spotipy
import spotipy.util as util

In [2]:
USERNAME = 'augt12ocf9csxa4s8kbq9reg8' #your spotify username
CLIENT_ID = 'b954b92b9a674e6a9cf7322649da82d1' #set at your developer account
CLIENT_SECRET = '920cd41023994b1a99210243ce8967b2' #set at your developer account
REDIRECT_URI = 'https://google.com/' #set at your developer account, usually "http://localhost:8000"
SCOPE = 'user-library-read' # or else
# ps. REDIRECT_URI is crucial here. if http://localhost:8000 is not set, or with a single '/' misplaced, it will generate a connection error.

# then pass them:

token = util.prompt_for_user_token(username = USERNAME, 
                                   scope = SCOPE, 
                                   client_id = CLIENT_ID, 
                                   client_secret = CLIENT_SECRET, 
                                   redirect_uri = REDIRECT_URI)

if token:
   sp = spotipy.Spotify(auth=token)



            User authentication requires interaction with your
            web browser. Once you enter your credentials and
            give authorization, you will be redirected to
            a url.  Paste that url you were directed to to
            complete the authorization.

        
Opened https://accounts.spotify.com/authorize?client_id=b954b92b9a674e6a9cf7322649da82d1&response_type=code&redirect_uri=https%3A%2F%2Fgoogle.com%2F&scope=user-library-read in your browser




Enter the URL you were redirected to:  https://www.google.com/?code=AQB6_guH_Q9DO3bhNcW534HrKK8bs0qvOZuuUDkWgYuzBXz-cdVz2uFdXlFM4X3nkXIfZ-yyx5-LVeSxJWtGp5j6cH3PhdznuNqVhfrh02QG0eF38xbO-UiPvtFg-lfr6FudpyjfwIQouNkiDbe6SfGR4yEM_MV7SBFEZ3FA7sWicsF7vXl38ekWg9Qui4D_Awk0pdBTDVkbFAwT


In [4]:
token

'BQAzm9VZMwr75qsRL2WkGWTyRVqFn9OYXIEFEzlOZfijT07_L8W5VBu0KymszA5Eg3n2_4np7yS6zibHQnfDt-DS8GZQ-7heWLAh51v4J15HzBCZhMdLJYZKgVq8aPaWU6Rfj52bmNPKnIUCvClGq-ixU2Laml5q_oUZwN7-sPko'

In [7]:
token_content = {'token': token}

In [8]:
type(token_content)

dict

In [9]:
token_content['token']

'BQAuxAqZlqyiBIPC-2MnrYjFH2amjQm8tLkh_ZYI_HkJ8MUHepG_lQrofpN--fGXM8b3tfXMfshOUvoF2Dkuh8iCepPvIvXqxCfrfSi52Rr8Njq0cnyLpde4_xo1v0-lMDjpMpeEOccK7yV_PPBVadskMoOr'

In [10]:
def get_id(token):
    sp = spotipy.Spotify(auth=token)
    results = sp.current_user_saved_tracks()
    song_id = results['items'][0]['track']['id']
    return song_id

In [11]:
def get_features(song_id):
    results_dict = sp.audio_features(song_id)[0]
    audio_features = {"audio_features":{key: results_dict[key] for key in results_dict.keys() & {'danceability', 'energy',
                                                                                                'key', 'loudness', 'mode',
                                                                                                'speechiness', 'acousticness',
                                                                                                'instrumentalness', 'liveness',
                                                                                                'valence', 'tempo', 'time_signature'}}}
        
    return audio_features

In [12]:
 acoustical_features = get_features(get_id(token))

In [13]:
acoustical_features

{'audio_features': {'liveness': 0.204,
  'acousticness': 0.472,
  'key': 9,
  'danceability': 0.889,
  'energy': 0.816,
  'speechiness': 0.247,
  'mode': 1,
  'loudness': -4.67,
  'valence': 0.777,
  'instrumentalness': 0,
  'time_signature': 4,
  'tempo': 96.056}}

# Flask Predict function

In [14]:
def predict(content):
    similar_songs = []
    print('Loading dataframe...')
    dataframe = pd.DataFrame.from_dict(
        json_normalize(content['audio_features']),
                                orient='columns')
    print('Dataframe Object Created')
    print('Loading pickled scaler...')
    scalar = load('scalar2.joblib')
    print('Pickled scaler loaded')
    print('Scaling dataframe object...')
    dataframe_scaled = scalar.transform(dataframe)
    print('Dataframe scaled')
    print('Loading pickled model...')
    model = load('model2.joblib')
    print('Model loaded')
    results = model.kneighbors([dataframe_scaled][0])[1]
    print('Prediction executed')
    print('song_id_list loading...')
    song_id_list = load('song_id_list2.joblib')
    print('song_id_list loaded')
    
    print('beginning for loop...')
    for song_row in results[0][1:]:
        song_id = song_id_list[song_row]
        similar_songs.append({'similarity': [.99], 'values': song_id})
    json_dict = {"songs": similar_songs}
    return json_dict


In [17]:
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from flask import request
from joblib import dump
from joblib import load

import pandas as pd
from pandas.io.json import json_normalize
from flask import jsonify

In [18]:
%timeit

predict(acoustical_features)

Loading dataframe...
Dataframe Object Created
Loading pickled scaler...
Pickled scaler loaded
Scaling dataframe object...
Dataframe scaled
Loading pickled model...
Model loaded
Prediction executed
song_id_list loading...
song_id_list loaded
beginning for loop...


{'songs': [{'similarity': [0.99], 'values': '5YdNeg0onlsEFoNdFjwG9X'},
  {'similarity': [0.99], 'values': '3XDPtaCpjC83LW1E7wfeVW'},
  {'similarity': [0.99], 'values': '6Ycon1DmLHH4xVVZaJqEdI'},
  {'similarity': [0.99], 'values': '6T8DFxrmY8y03FGYDxkmpg'},
  {'similarity': [0.99], 'values': '0lO6VPXcW9wIOKxZ0wecjG'},
  {'similarity': [0.99], 'values': '56KyV36puztkiJ62ca3D1t'},
  {'similarity': [0.99], 'values': '2UUIpe2Jtflo8dX9jeTSqQ'},
  {'similarity': [0.99], 'values': '6lneyspCLiV4hfccAidPeU'},
  {'similarity': [0.99], 'values': '4nKjzgBwU9ksBUG4VJ787c'},
  {'similarity': [0.99], 'values': '6galGfTFHHzsPieYkReYyN'},
  {'similarity': [0.99], 'values': '7F9Dbj9ul1FtOPcsXDF3R9'},
  {'similarity': [0.99], 'values': '0e7jY6MjiJdKNwUiwDhyxM'},
  {'similarity': [0.99], 'values': '1LVfv8UzT0Zw3YjAbfcl4U'},
  {'similarity': [0.99], 'values': '3QUgFGZvPhtdioo1MRepuE'},
  {'similarity': [0.99], 'values': '6SuyOjhiF6vssCI7URWLDZ'},
  {'similarity': [0.99], 'values': '015p8TgFahkz0viuux9YGJ'},

# Dump

In [12]:
dump(model, 'model2.joblib', compress=True)

['model2.joblib']

In [4]:
# %time

# pickle_loaded = pickle.load(open('Flask_API/SOUNDDRIP/models/knn_model_v1.pkl', 'rb'))
# joblib_loaded = load('model.joblib')



In [5]:
# dump(data_scaled, 'data_scaled.joblib', compress=True)

In [13]:
dump(scaler, 'scalar2.joblib', compress=True)

['scalar2.joblib']

In [18]:
dump(song_id_list, 'song_id_list2.joblib', compress=True)

['song_id_list2.joblib']